In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,29929
2,Huelva,Confirmados PDIA 14 días,1220
3,Huelva,Tasa PDIA 14 días,"237,73798156556308"
4,Huelva,Confirmados PDIA 7 días,610
5,Huelva,Total Confirmados,30106
6,Huelva,Curados,27034
7,Huelva,Fallecidos,375


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  29929.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  7825.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 866 personas en los últimos 7 días 

Un positivo PCR cada 393 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,29929.0,610.0,1220.0,513170.0,118.868991,237.737982,104.0
Huelva-Costa,17316.0,259.0,582.0,289548.0,89.449763,201.002943,52.0
Condado-Campiña,9621.0,290.0,545.0,156231.0,185.622572,348.842419,38.0
Huelva (capital),7825.0,166.0,366.0,143837.0,115.408414,254.454695,24.0
Sierra de Huelva-Andévalo Central,2651.0,61.0,93.0,67391.0,90.516538,138.000623,14.0
Almonte,1696.0,62.0,146.0,24507.0,252.988942,595.748154,10.0
Palma del Condado (La),1098.0,46.0,59.0,10801.0,425.886492,546.245718,7.0
Moguer,1188.0,55.0,80.0,21867.0,251.520556,365.848082,7.0
Valverde del Camino,520.0,31.0,44.0,12750.0,243.137255,345.098039,6.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Galaroza,41.0,4.0,11.0,1382.0,289.435601,795.947902,0.0
Bollullos Par del Condado,926.0,46.0,92.0,14387.0,319.733092,639.466185,3.0
Almonte,1696.0,62.0,146.0,24507.0,252.988942,595.748154,10.0
Palma del Condado (La),1098.0,46.0,59.0,10801.0,425.886492,546.245718,7.0
Cabezas Rubias,34.0,2.0,3.0,706.0,283.286119,424.929178,NaN
Moguer,1188.0,55.0,80.0,21867.0,251.520556,365.848082,7.0
Punta Umbría,770.0,25.0,56.0,15355.0,162.813416,364.702051,5.0
Escacena del Campo,154.0,3.0,8.0,2287.0,131.176213,349.803236,0.0
Condado-Campiña,9621.0,290.0,545.0,156231.0,185.622572,348.842419,38.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Rociana del Condado,535.0,4.0,19.0,7939.0,50.384179,239.324852,1.0,0.210526
Ayamonte,1300.0,4.0,13.0,21104.0,18.953753,61.599697,2.0,0.307692
Bonares,261.0,2.0,6.0,6060.0,33.003300,99.009901,0.0,0.333333
Lepe,2105.0,8.0,22.0,27880.0,28.694405,78.909613,2.0,0.363636
Galaroza,41.0,4.0,11.0,1382.0,289.435601,795.947902,0.0,0.363636
Escacena del Campo,154.0,3.0,8.0,2287.0,131.176213,349.803236,0.0,0.375000
Gibraleón,605.0,7.0,18.0,12737.0,54.957996,141.320562,2.0,0.388889
Palos de la Frontera,610.0,9.0,23.0,11742.0,76.647931,195.878045,2.0,0.391304
Beas,204.0,2.0,5.0,4341.0,46.072334,115.180834,0.0,0.400000
